### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Analysis

1. Across the entire district, the mean math and reading scores had no significant change between school years. Over the four years, the district-wide math average stayed ~80 and the reading average stayed ~82.5. 

2. Schools with per-student budgets less than $615 far out-performed schools with higher budgets in the percentage of students passing both reading and math by a margin of around 30%.

3. Similarly, schools with less than 2500 students beat out schools with larger student bodies in the percentage of students passing both reading and math by almost 40%.

4. District schools are not far behind Charter schools in the number of students passing reading. Charter schools did however largely beat district schools in the number of students passing math and students passing both subjects.

In [289]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [268]:
school_data_complete.columns

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

In [269]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [270]:
def district_summary(): 

    total_schools = school_data["School ID"].shape[0]
    total_students = school_data["size"].sum()
    total_budget = school_data["budget"].sum()

    avg_math = school_data_complete["math_score"].mean()
    avg_read = school_data_complete["reading_score"].mean()

    pass_math_count = school_data_complete.query("math_score >= 70").count()["student_name"]
    pass_read_count = school_data_complete.query("reading_score >= 70").count()["student_name"]
    pass_both_count = school_data_complete.query("(math_score >= 70) & (reading_score >= 70)").count()["student_name"]

    pass_math_perc = 100*pass_math_count/float(total_students)
    pass_read_perc = 100*pass_read_count/float(total_students)
    pass_both_perc = 100*pass_both_count/float(total_students)

    district_df = pd.DataFrame({"Total # of Schools": [total_schools], "Total # of Students": ["{:,}".format(total_students)], "Total Budget": ["${:,.2f}".format(total_budget)], "Average Math Score": [avg_math], "Average Reading Score": [avg_read], "% of Students Passing Math": ["{:.2f}%".format(pass_math_perc)], "% of Students Passing Reading": ["{:.2f}%".format(pass_read_perc)], "% of Students Passing Overall": ["{:.2f}%".format(pass_both_perc)]})

    return district_df

In [271]:
district_summary()

,Total # of Schools,Total # of Students,Total Budget,Average Math Score,Average Reading Score,% of Students Passing Math,% of Students Passing Reading,% of Students Passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [272]:
def school_summary():

    grouped = school_data_complete.groupby(["school_name"])

    school_types = school_data.set_index(["school_name"])["type"]

    student_count = school_data_complete["school_name"].value_counts()
    budget_count = grouped.mean()["budget"]
    capita_budget = budget_count / student_count

    avg_math = grouped.mean()["math_score"]
    avg_read = grouped.mean()["reading_score"]

    math_group = school_data_complete.query("math_score >= 70").groupby(["school_name"])
    read_group = school_data_complete.query("reading_score >= 70").groupby(["school_name"])
    both_group = school_data_complete.query("(math_score >= 70) & (reading_score >= 70)").groupby(["school_name"])

    pass_math_count = math_group.count()["student_name"]
    pass_read_count = read_group.count()["student_name"]
    pass_both_count = both_group.count()["student_name"]

    pass_math_perc = 100*pass_math_count / student_count
    pass_read_perc = 100*pass_read_count / student_count
    pass_both_perc = 100*pass_both_count / student_count

    schools_df = pd.DataFrame({"School Type": school_types, "Total # of Students": student_count.map("{:,}".format), "Total Budget": budget_count.map("${:,.2f}".format), "Budget per Student": capita_budget.map("${:,.2f}".format), "Average Math Score": avg_math, "Average Reading Score": avg_read, "% of Students Passing Math": pass_math_perc.map("{:.2f}%".format), "% of Students Passing Reading":  pass_read_perc.map("{:.2f}%".format), "% of Students Passing Overall":  pass_both_perc.map("{:.2f}%".format)})

    return schools_df
    

In [273]:
school_summary()

,School Type,Total # of Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% of Students Passing Math,% of Students Passing Reading,% of Students Passing Overall
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [274]:
school_summary().sort_values("% of Students Passing Overall",ascending=False).head(5)

,School Type,Total # of Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% of Students Passing Math,% of Students Passing Reading,% of Students Passing Overall
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [275]:
school_summary().sort_values("% of Students Passing Overall").head(5)

,School Type,Total # of Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% of Students Passing Math,% of Students Passing Reading,% of Students Passing Overall
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [276]:
def by_grade(grade):
    grade_df = school_data_complete.loc[school_data_complete["grade"]==grade]
    return grade_df

def grade_summary(subject):
    grades = ["9th","10th","11th","12th"]
    grade_dict = dict.fromkeys(grades)
    for grade in grades:
        grade_grouped = by_grade(grade).groupby(["school_name"])
        grade_series = grade_grouped.mean()[f'{subject}_score']
        grade_dict[grade] = grade_series
    grade_df = pd.DataFrame(grade_dict)
    grade_df.index.name = "School Name"
    return grade_df

In [277]:
grade_summary('math')

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [312]:
grade_summary('math').mean()

9th     80.351617
10th    80.378948
11th    80.575873
12th    80.423811
dtype: float64

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [278]:
grade_summary('reading')

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [313]:
grade_summary('reading').mean()

9th     82.513318
10th    82.505439
11th    82.559485
12th    82.554817
dtype: float64

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [279]:
min_bin = school_summary()["Budget per Student"].min()
max_bin = school_summary()["Budget per Student"].max()

In [280]:
def replacedollar(string):
    return float(string.replace('$',''))
def replaceperc(string):
    return float(string.replace('%',''))
def replacecomma(string):
    return float(string.replace(',',''))

In [281]:
def budget_bins():
    bins = [0, 585, 615, 645, 675]
    bin_names = ["<$585", "$585-615", "$615-645", "$645-675"]
    schoolsdf = school_summary()
    schoolsdf["Spending Range (per Student)"] = pd.cut(school_summary()["Budget per Student"].map(replacedollar),bins=bins,labels=bin_names)
    schoolsdf["% of Students Passing Math"] = schoolsdf["% of Students Passing Math"].map(replaceperc)
    schoolsdf["% of Students Passing Reading"] = schoolsdf["% of Students Passing Reading"].map(replaceperc)
    schoolsdf["% of Students Passing Overall"] = schoolsdf["% of Students Passing Overall"].map(replaceperc)
    binned_groupavg = schoolsdf.groupby(["Spending Range (per Student)"]).mean()
    binned_avgmath = binned_groupavg["Average Math Score"]
    binned_avgread = binned_groupavg["Average Reading Score"]
    binned_passmath = binned_groupavg["% of Students Passing Math"].map("{:.2f}%".format)
    binned_passread = binned_groupavg["% of Students Passing Reading"].map("{:.2f}%".format)
    binned_overall = binned_groupavg["% of Students Passing Overall"].map("{:.2f}%".format)
    budget_df = pd.DataFrame({"Average Math Score": binned_avgmath, "Average Reading Score": binned_avgread, "% of Students Passing Math": binned_passmath, "% of Students Passing Reading": binned_passread, "% of Students Passing Overall": binned_overall})
    return budget_df

In [282]:
budget_bins()

,Average Math Score,Average Reading Score,% of Students Passing Math,% of Students Passing Reading,% of Students Passing Overall
Spending Range (per Student),,,,,
<$585,83.455399,83.933814,93.46%,96.61%,90.37%
$585-615,83.599686,83.885211,94.23%,95.90%,90.22%
$615-645,79.079225,81.891436,75.67%,86.11%,66.11%
$645-675,76.997210,81.027843,66.16%,81.13%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [283]:
def size_bins():
    bins = [0, 1500, 2500, 4000, 5000]
    bin_names = ["Small (<1500)", "Medium (1500-2500)", "Large (2500-4000)", "Extra-Large (>4000)"]
    schoolsdf = school_summary()
    schoolsdf["School Size"] = pd.cut(school_summary()["Total # of Students"].map(replacecomma),bins=bins,labels=bin_names)
    schoolsdf["% of Students Passing Math"] = schoolsdf["% of Students Passing Math"].map(replaceperc)
    schoolsdf["% of Students Passing Reading"] = schoolsdf["% of Students Passing Reading"].map(replaceperc)
    schoolsdf["% of Students Passing Overall"] = schoolsdf["% of Students Passing Overall"].map(replaceperc)
    binned_groupavg = schoolsdf.groupby(["School Size"]).mean()
    binned_avgmath = binned_groupavg["Average Math Score"]
    binned_avgread = binned_groupavg["Average Reading Score"]
    binned_passmath = binned_groupavg["% of Students Passing Math"].map("{:.2f}%".format)
    binned_passread = binned_groupavg["% of Students Passing Reading"].map("{:.2f}%".format)
    binned_overall = binned_groupavg["% of Students Passing Overall"].map("{:.2f}%".format)
    size_df = pd.DataFrame({"Average Math Score": binned_avgmath, "Average Reading Score": binned_avgread, "% of Students Passing Math": binned_passmath, "% of Students Passing Reading": binned_passread, "% of Students Passing Overall": binned_overall})
    return size_df

In [284]:
size_bins()

,Average Math Score,Average Reading Score,% of Students Passing Math,% of Students Passing Reading,% of Students Passing Overall
School Size,,,,,
Small (<1500),83.664898,83.892148,93.50%,96.45%,90.12%
Medium (1500-2500),83.359224,83.898984,93.69%,96.67%,90.62%
Large (2500-4000),76.821621,80.957921,66.59%,80.39%,53.50%
Extra-Large (>4000),77.136883,80.978256,66.50%,81.34%,53.90%


## Scores by School Type

* Perform the same operations as above, based on school type

In [287]:
def type_bins():
    schoolsdf = school_summary()
    schoolsdf["% of Students Passing Math"] = schoolsdf["% of Students Passing Math"].map(replaceperc)
    schoolsdf["% of Students Passing Reading"] = schoolsdf["% of Students Passing Reading"].map(replaceperc)
    schoolsdf["% of Students Passing Overall"] = schoolsdf["% of Students Passing Overall"].map(replaceperc)
    binned_groupavg = schoolsdf.groupby(["School Type"]).mean()
    binned_avgmath = binned_groupavg["Average Math Score"]
    binned_avgread = binned_groupavg["Average Reading Score"]
    binned_passmath = binned_groupavg["% of Students Passing Math"].map("{:.2f}%".format)
    binned_passread = binned_groupavg["% of Students Passing Reading"].map("{:.2f}%".format)
    binned_overall = binned_groupavg["% of Students Passing Overall"].map("{:.2f}%".format)
    type_df = pd.DataFrame({"Average Math Score": binned_avgmath, "Average Reading Score": binned_avgread, "% of Students Passing Math": binned_passmath, "% of Students Passing Reading": binned_passread, "% of Students Passing Overall": binned_overall})
    return type_df

In [288]:
type_bins()

,Average Math Score,Average Reading Score,% of Students Passing Math,% of Students Passing Reading,% of Students Passing Overall
School Type,,,,,
Charter,83.473852,83.896421,93.62%,96.59%,90.43%
District,76.956733,80.966636,66.55%,80.80%,53.67%
